In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'  # disable oneDNN custom operations

import warnings
warnings.filterwarnings('ignore')

import logging
logging.getLogger('tensorflow').setLevel(logging.ERROR)

In [2]:
from romgw.config.constants import PROJECT_ROOT, MODE_VALUES
from romgw.config.types import BBHSpinType, DatasetType, ModeType, ComponentType, ModelNameType
from romgw.config.validation import validate_literal, validate_dependent_literal
from romgw.deep_learning.hyper import HyperParams

In [ ]:
bbh_spin = "NS"
dataset = "train_xl"
mode = "2,2"
component = "amplitude"
model_name = "NonLinRegV1"

In [10]:
bbh_spin = validate_literal(
    value=bbh_spin,
    literal_type=BBHSpinType,
)
dataset = validate_literal(
    value=dataset,
    literal_type=DatasetType,
)
mode = validate_dependent_literal(
    value=mode,
    literal_type=ModeType,
    parent_value=bbh_spin,
    parent_literal_type=BBHSpinType,
    dependency_map=MODE_VALUES,
)
component = validate_literal(
    value=component,
    literal_type=ComponentType,
)
model_name = validate_literal(
    value=model_name,
    literal_type=ModelNameType,
)

In [11]:
model_dir = (
    PROJECT_ROOT / "data" / bbh_spin / dataset / mode / component
    / "models" / model_name
)
if not model_dir.is_dir():
    raise NotADirectoryError(f"Could not find the directory {model_dir}")

In [12]:
# Load and view HyperParams
hyper = HyperParams.from_dir(model_dir)
print(hyper)

HyperParams:
- optimiser: epsilon=1e-06, learning_rate=0.03
- lrop: factor=0.5, patience=32, min_lr=1e-10
- early_stopping: patience=256
- fit: epochs=1000, batch_size=256


In [14]:
# Make and save new HyperParams
optimiser_hp = {
    "epsilon": 1e-6,
    "learning_rate": 5e-2,
}
lrop_hp = {
    "factor": 0.5,
    "patience": 64,
    "min_lr": 1e-10,
}
early_stopping_hp = {
    "patience": 256,
}
fit_hp = {
    "epochs": 1000,
    "batch_size": 256,
}

hyper = HyperParams(optimiser_hp, lrop_hp, early_stopping_hp, fit_hp)
hyper.to_dir(model_dir)

In [15]:
# Load and view HyperParams
hyper = HyperParams.from_dir(model_dir)
print(hyper)

HyperParams:
- optimiser: epsilon=1e-06, learning_rate=0.05
- lrop: factor=0.5, patience=64, min_lr=1e-10
- early_stopping: patience=256
- fit: epochs=1000, batch_size=256
